In [1]:
from playwright.async_api import async_playwright, Page, ElementHandle

In [2]:
pw = await async_playwright().start()
browser = await pw.chromium.launch(headless=False)
page = await browser.new_page()
await page.goto("https://www.google.com")

<Response url='https://www.google.com/' request=<Request url='https://www.google.com/' method='GET'>>

In [3]:
# Accept cookies
accept_button = await page.wait_for_selector("button#L2AGLb")
await accept_button.click()

In [4]:
def get_zone_url(zone: str) -> str:
    base_url = f"https://www.google.com/search?q=bares+y+restaurantes+bilbao+{zone}"
    sit_url = "&uds=AMwkrPv4b_4cYRjioaEQc4CO9do7Uwi_qVOE8vzTSEbjWaJqe_OdBFIUriiQ76JC9goO_6Bx5llttfasDMCpy0gSYR6-XrEKD9DJEJUU3WEPxZa7Gn-Iu4A3f6H61I2ZIL5WyL3yJBLXCFix-ukO6c1PbOlN_mSAvg&udm=1&sa=X&ved=2ahUKEwjA64Pj1cmEAxW70wIHHf0BDl0Qs6gLegQICxAB&biw=1280&bih=720#vhid=/g/1td66mxk&vssid=rllrl&ip=1"

    return base_url + sit_url


await page.goto(get_zone_url("deusto"))

<Response url='https://www.google.com/search?q=bares+y+restaurantes+bilbao+deusto&uds=AMwkrPv4b_4cYRjioaEQc4CO9do7Uwi_qVOE8vzTSEbjWaJqe_OdBFIUriiQ76JC9goO_6Bx5llttfasDMCpy0gSYR6-XrEKD9DJEJUU3WEPxZa7Gn-Iu4A3f6H61I2ZIL5WyL3yJBLXCFix-ukO6c1PbOlN_mSAvg&udm=1&sa=X&ved=2ahUKEwjA64Pj1cmEAxW70wIHHf0BDl0Qs6gLegQICxAB&biw=1280&bih=720' request=<Request url='https://www.google.com/search?q=bares+y+restaurantes+bilbao+deusto&uds=AMwkrPv4b_4cYRjioaEQc4CO9do7Uwi_qVOE8vzTSEbjWaJqe_OdBFIUriiQ76JC9goO_6Bx5llttfasDMCpy0gSYR6-XrEKD9DJEJUU3WEPxZa7Gn-Iu4A3f6H61I2ZIL5WyL3yJBLXCFix-ukO6c1PbOlN_mSAvg&udm=1&sa=X&ved=2ahUKEwjA64Pj1cmEAxW70wIHHf0BDl0Qs6gLegQICxAB&biw=1280&bih=720' method='GET'>>

In [5]:
from typing import Any


async def get_normalize_content(element: ElementHandle) -> str:
    if element is None:
        return None

    element_content = await element.inner_text()

    return (
        element_content.replace(",", "")
        .replace("(", "")
        .replace(")", "")
        .replace("\xa0", "")
        .replace("mil", "000")
    )


async def get_element_content(page: Page, element: ElementHandle) -> dict[str, Any]:
    # Getting card content
    title = await element.query_selector("span.OSrXXb.e62wic")
    show_more = await element.query_selector("div[jsname='fsEqyc']")

    await title.click()
    if show_more:
        await show_more.click()

    # Getting card details
    stars_avg = await element.query_selector("span.yi40Hd.YrbPuc")
    n_reviews = await element.query_selector("span.RDApEe.YrbPuc")
    price_range = await element.query_selector("div:nth-child(2) > span:nth-child(2)")
    description = await page.query_selector("c-wiz[jsrenderer='Gdjwac']")
    facilities = await page.query_selector("div.fgzP8e")
    street = await page.query_selector("div.F2yIXb")
    phone_number = await page.query_selector("div.eigqqc")

    return {
        "title": await get_normalize_content(title),
        "description": await get_normalize_content(description),
        "n_reviews": await get_normalize_content(n_reviews),
        "stars_avg": await stars_avg.inner_text(),
        "price_range": await get_normalize_content(price_range),
        "facilities": await get_normalize_content(facilities),
        "street": await street.inner_text(),
        "phone_number": await get_normalize_content(phone_number),
    }

In [6]:
list_of_restaurants = await page.query_selector_all("div.rllt__details")

In [16]:
button = await page.query_selector("div[jsname='fsEqyc']")

await button.click()

In [7]:
restaurant_data = []
for restaurant in list_of_restaurants[:20]:
    data = await get_element_content(page, restaurant)

    if data["title"] is None:
        continue

    restaurant_data.append(data)
    await page.wait_for_timeout(3000)
    break

restaurant_data

{'title': 'Restaurante Deustoarrak',
 'description': 'Cafetería y restaurante con piano sofá corrido y muebles '
                'vintage que sirve pintxos y menú de recetas vascas.',
 'n_reviews': '19000',
 'stars_avg': '4,1',
 'price_range': '10-20€',
 'facilities': 'Tiene terraza· Tiene comedor privado· Sirve buenos cócteles',
 'street': 'Avenida de, Madariaga Etorbidea, 9, Deusto, 48014 Bilbo, Bizkaia',
 'phone_number': None}
-----


[{'title': 'Restaurante Deustoarrak',
  'description': 'Cafetería y restaurante con piano sofá corrido y muebles vintage que sirve pintxos y menú de recetas vascas.',
  'n_reviews': '19000',
  'stars_avg': '4,1',
  'price_range': '10-20€',
  'facilities': 'Tiene terraza· Tiene comedor privado· Sirve buenos cócteles',
  'street': 'Avenida de, Madariaga Etorbidea, 9, Deusto, 48014 Bilbo, Bizkaia',
  'phone_number': None}]

In [78]:
import pandas as pd

df_restaurant = pd.DataFrame(restaurant_data)
df_restaurant

,title,description,n_reviews,stars_avg,price_range,facilities,street,phone_number
0,El plaza de indautxu,None,394,"3,9",10-20€,Tiene terraza· Sillas altas disponibles,"Santimami Zumarkalea, 46, Abando, 48010 Bilbo,...",946 00 03 41
1,Bar Mugi,Restaurante de gestión fa000iar que ofrece tap...,13000,"4,4",10-20€,Tiene terraza· Tiene comedor privado· Sillas a...,"Poza Lizentziatuaren Kalea, 55, Abando, 48013 ...",944 41 30 16
2,Bodega Indautxu,None,643,"4,3",1-10€,Tiene terraza· Se sirve comida en el bar,"Gregorio de la Revilla Zumarkalea, 18, Abando,...",944 42 28 23
3,Patio Bilbao,None,301,"4,1",10-20€,Tiene terraza· Tiene Wi-Fi· Sillas altas dispo...,"Kalea Manuel Allende, 21, Abando, 48010 Bilbo,...",946 07 52 76
4,Bar Joserra,None,587,"4,3",1-10€,Tiene terraza· Se sirve comida en el bar,"Indautxu Kalea, 4, Abando, 48011 Bilbo, Bizkaia",944 42 22 35
5,Ikatza Bar Restaurante,None,365,"4,3",10-20€,Tiene terraza· Sirve buenos cócteles· Ideal pa...,"Simón Bolívar Kalea, 25, Abando, 48010 Bilbo, ...",946 41 43 88
6,Txirene | 100% Bilbao,None,551,"4,4",None,Sirve buenos cócteles,"Poza Lizentziatuaren Kalea, 26, Abando, 48009 ...",944 55 46 88
7,Restaurante Egaña Bar,None,496,"4,2",€,Tiene terraza· Ideal para ver deportes· Tiene ...,"Egaña Kalea, 14, Abando, 48010 Bilbo, Bizkaia",685 92 95 84
8,Zaka Restaurante,Restaurante a la última decorado con un estilo...,26000,"4,3",20-30€,Tiene terraza· Sirve buenos cócteles· Ideal pa...,"Simón Bolívar Kalea, 11, Abando, 48011 Bilbo, ...",944 01 45 61
9,El Huevo Berria,Decoración del Athletic Club y barra de pintxo...,13000,"4,1",1-10€,Tiene terraza· No se aceptan reservas,"Poza Lizentziatuaren Kalea, 65, Abando, 48013 ...",944 27 13 48
